In [1]:
import requests
import json
import re

In [2]:
URL = "https://www.radiofrance.fr"
url_affaires = "https://www.radiofrance.fr/franceculture/podcasts/affaires-etrangeres"
url_concordance = "https://www.radiofrance.fr/franceculture/podcasts/concordance-des-temps"
url_culture = "https://www.radiofrance.fr/franceculture/podcasts/cultures-monde"
url_eco = "https://www.radiofrance.fr/franceculture/podcasts/entendez-vous-l-eco"

In [119]:
folder_path_affaires = "data/affaires_etrangeres"
folder_path_concordance = "data/concordance_des_temps"
folder_path_culture = "data/culture_monde"
folder_path_eco = "data/entendez_eco"

In [3]:
from bs4 import BeautifulSoup

In [6]:
def episode_list(podcast_url):
    page = requests.get(podcast_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all("div", class_="CardDetails-title svelte-690u24")
    list_ep = []
    for result in results:
        link = result.find("a")
        episode_url = URL + link.attrs["href"]
        list_ep.append(episode_url)
    return list_ep

ep_list_concordance = episode_list(url_concordance)


In [108]:
episode_page = requests.get(ep_list_concordance[0])
ep_list_concordance[2]

'https://www.radiofrance.fr/franceculture/podcasts/concordance-des-temps/sites-de-rencontre-et-petites-annonces-une-perennite-0-3409483'

In [102]:
def file_name(episode_url):
    string = re.search(r"([^\/]+$)", episode_url).group()
    res = string.strip("-0123456789")+".mp3"
    return res

In [104]:
file_name(ep_list_concordance[2])

'sites-de-rencontre-et-petites-annonces-une-perennite.mp3'

In [83]:

def episode_mp3_list(episode_url):
    episode_page = requests.get(episode_url)# ep_list_concordance[0])
    soup = BeautifulSoup(episode_page.content, 'html.parser')
    results = soup.find_all(type="application/ld+json")
    mp3_list = []
    for res in results:
        if "mp3" in res.text:
            json_dict= json.loads("".join(res))
            ep_mp3_url = json_dict['@graph'][0]["mainEntity"]["contentUrl"]
            mp3_list.append(ep_mp3_url)
    return mp3_list
        

In [105]:
mp3_url_list = episode_mp3_list(ep_list_concordance[2])
name = file_name(ep_list_concordance[2])

In [88]:
r = requests.get(mp3_url_list[0], allow_redirects=True)
print(r.headers.get('content-type'))


audio/mpeg


In [ ]:
import logging

In [120]:
def download_mp3(list_episodes:list, folder_path:str):
        for episode_url in list_episodes:
                episode_mp3 = episode_mp3_list(episode_url)[0]
                r = requests.get(episode_mp3, allow_redirects=True)
                name = file_name(episode_url)
                if r.headers.get('content-type')=="audio/mpeg":
                        with open(folder_path+"/"+name, 'wb') as f:
                                f.write(r.content)
                else:
                        print(f"No audio/mpeg file was downloaded for {name}!")

In [121]:
download_mp3(ep_list_concordance,folder_path_concordance)

FileNotFoundError: [Errno 2] No such file or directory: 'data/concordance_des_temps/la-silhouette-aux-origines-d-une-obsession.mp3'